In [1]:
import torch
from resnet.resnet import ResNet18
import torchvision
from torchvision import datasets, transforms
import numpy as np
import torch.nn as nn

from itertools import product

(11164362,)
(11164362,)


In [2]:
device = 'cuda'

In [3]:
net_sgd = torch.nn.DataParallel(ResNet18())
net_sgd2 = torch.nn.DataParallel(ResNet18())

In [4]:
checkpoint = torch.load('resnet/checkpoint/Resnet18_optim_sgd_lr_1e-2_wd_5e-4_seed_1_batch_128.t7')
net_sgd.load_state_dict(checkpoint['net'])
#checkpoint = torch.load('resnet/checkpoint/optim_adam_lr_1e-4_wd_5e-4_seed_1.t7')
#net_sgd2.load_state_dict(checkpoint['net'])

In [10]:
fixed_transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

randomed_transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

fixed_trainset = torchvision.datasets.CIFAR10(root='~/data/cifar10', train=True, download=True, transform=fixed_transform_train)
fixed_trainloader = torch.utils.data.DataLoader(fixed_trainset, batch_size=100, shuffle=False, num_workers=2)

trainset = torchvision.datasets.CIFAR10(root='~/data/cifar10', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=False, num_workers=2)


testset = torchvision.datasets.CIFAR10(root='~/data/cifar10', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=1000, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [6]:
loss_function = nn.CrossEntropyLoss()

In [7]:
def test(net, loader):
    net.eval()
    correct = 0
    total =  0
    test_loss = []
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = loss_function(outputs,targets)
            test_loss.append(loss.item())
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            
    acc = 100.*correct/total
    test_loss_mean = np.mean(test_loss)
    test_loss_std = np.std(test_loss)
    return test_loss_mean, test_loss_std, acc

In [8]:
net_sgd = net_sgd.module
#net_sgd2 = net_sgd2.module


In [11]:
print('Train loss mean: {:.6f} std: {:.6f} acc:{:.2f}'.format(*test(net_sgd, trainloader)))
print('Fixed Train loss mean: {:.6f} std: {:.6f} acc:{:.2f}'.format(*test(net_sgd, fixed_trainloader)))
print('Test loss mean: {:.6f} std: {:.6f} acc:{:.2f}'.format(*test(net_sgd, testloader)))

Train loss mean: 0.069188 std: 0.031851 acc:97.73
Fixed Train loss mean: 0.057461 std: 0.028480 acc:98.12
Test loss mean: 0.209715 std: 0.024890 acc:93.27


In [12]:
tnet = ResNet18().to(device)


In [13]:
zero_vector = np.zeros(tnet.get_weight_vector().shape)
zero_vector.shape

(11164362,)

# Filterwise Normalization

In [14]:
b1, b2 = np.random.normal(loc = zero_vector, scale=1),np.random.normal(loc = zero_vector, scale=1)
b1, b2 = tnet.filterwisely_normalize(b1), tnet.filterwisely_normalize(b2)

In [15]:
origin = net_sgd.get_weight_vector()

# 2D Visualization

In [16]:
r = 1.0
scale=11

In [17]:
x = np.linspace(-r, r, scale)
y = np.linspace(-r, r,scale)
nx,ny = len(x), len(y)
nx,ny

(11, 11)

In [22]:
loss_map = np.zeros((nx,ny))
acc_map = np.zeros((nx,ny))
std_map = np.zeros((nx,ny))

In [19]:
tt = origin + b1*0
tnet.set_weight_vector(tt)
test(tnet,testloader)

(2.3057875871658324, 0.003304418509658315, 10.0)

In [23]:

for i,j in product(range(nx),range(ny)):
    vec = origin + b1 * x[i] + b2 * y[j]
    tnet.set_weight_vector(vec)
    loss, std, acc = test(tnet, trainloader)
    loss_map[i, j] = loss
    acc_map[i, j] = acc
    std_map[i, j] = std
    print(i, j, x[i], y[j], acc, loss)

0 0 -1.0 -1.0 11.424 2.6607245330810545
0 1 -1.0 -0.8 10.088 2.5811072783470155


Process Process-17:
Process Process-18:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/leesy714/venv/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/home/leesy714/venv/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeout):
  File

KeyboardInterrupt: 

In [ ]:
from matplotlib import pyplot as plt
import matplotlib.colors as colors
%matplotlib inline

In [ ]:
X,Y = np.meshgrid(x,y)

X.shape

In [ ]:
loss_map.shape

In [ ]:
plt.figure(figsize=(6,6))
cs = plt.contourf(X,Y,loss_map,levels=100  )
cbar = plt.colorbar(cs)
#plt.xlim([0,b1_n])
#plt.ylim([0,b2_n])

In [ ]:
loss_map.shape